# IBM Data science Capstone Project

The battle of neighbourhoods is a cpastone data science project as a part of the IBM data science professional certificate and sevaral other specializations. 

This Project deals with Segmenting and clustering of the Neighbourhoods in Toronto.



## Segmenting and clustering Neighbourhoods in Toronto 

## Table of Contents

 <div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#about_dataset">About the dataset</a></li>
        <li><a href="#preprocessing">Data preprocessing</a></li>
        <li><a href="#ll">Latitude & Longitude </a></li>
        <li><a href="#explore">Explore the Data</a></li>
    </ol>
</div>

___
<div id="#about_dataset">
    <h2>About the dataset</h2>
</div>

___

- The data is colletted from the [wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) page

- This page contains a list of postal codes in canada where the first letter is M.



**Installing Required packages**

In [0]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install requests

**Importing Dependancies and packages**

In [0]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [0]:
# Get the HTML from the wiki page 
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_text = requests.get(wiki_url).text

Now we use BeautifulSoup module to extract the table data from the HTML text

In [28]:
# Extract the table data using BeautifulSoup
soup = BeautifulSoup(html_text)
table = soup.find('table', attrs={'class':'wikitable sortable'})
trs = table.find_all('tr')
print("Number of rows in the table(including columns header): ", len(trs))

# Extract the text from all the table cells and add all rows to a list of rows.
rows = list()
for tr in trs:
    td = tr.find_all('td')
    row = [ele.text.strip() for ele in td]
    if row:
        # Ignore empty rows with no 'td',
        # applicable for the column headers row.
        rows.append(row)

print("Number of rows with data in the table: ", len(rows))

Number of rows in the table(including columns header):  181
Number of rows with data in the table:  180


____
<div id="#preprocessing">
    <h2><b>Preprocessing</b></h2>
</div>

___

**Creating the pandas dataframe for neighborhood data**

Convert the data to a pandas dataframe with columns _postalCode_, _Borough_, and _Neighborhood_


In [29]:
df = pd.DataFrame(rows, columns=['PostalCode' ,'Borough', 'Neighborhood'])
print("Dataframe shape", df.shape)
df.head()

Dataframe shape (180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Remove rows with column values 'Not Assigned' for column 'Borough'

In [30]:
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True, drop=True)
print("Dataframe shape: ", df.shape)
df.head()

Dataframe shape:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


If a cell has a borough but not assigned neighborhood, then the neighborhood will be the same as the borough. So, replace 'Neighborhood' columns with value as 'Not assigned' to 'Borough'

In [31]:
df['Neighborhood'] = df.apply(
    lambda row: 
    row['Borough'] if row['Neighborhood'] == 'Not assigned' 
    else row['Neighborhood'],
    axis=1)
print("Dataframe shape: ", df.shape)
df.head()

Dataframe shape:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


More than one neighborhood can exist in one postal code area. Combine rows with the same postal code into a single row. For example, in the table on the Wikipedia page, M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [32]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].\
    apply(', '.join).to_frame()
df.reset_index(inplace=True)
print("The shape of the dataframe is: ", df.shape)
df.head(10)

The shape of the dataframe is:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


___
<div id="#ll">
    <h2><b>latitude & longitude</b></h2>
</div>

___

Now we add the latitude and longitude data to the DataFrame.

We will be using [the following lat & long data]( https://cocl.us/Geospatial_data) to get the data



In [0]:
# read the CSV file to pandas Data Frame
geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [34]:
# Data Exploration
print("Geospatial dataframe's shape: ", geospatial_data.shape)
geospatial_data.head()

Geospatial dataframe's shape:  (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The size of 103 rows matches that our original dataframe 'df'.

**Combining the two data frames**
 
 Let us combine the two dataframes such that the latiude and longitude values from ```geospatial_data``` are assigned to their corresponding rows in our original data frame  ```df```

In [35]:
# Concatenate the two dataframes using column "PostalCode" from 'df' and
# "Postal Code" from 'geospatial_data'.
df = pd.concat(
    [df.set_index('PostalCode'), geospatial_data.set_index('Postal Code')],
    axis=1, join='inner')
df.head()

,Borough,Neighborhood,Latitude,Longitude
M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Postal code has been set as the index, change it to a non-index column and reset index and renaming the ```index``` column to ```PostalCode```

In [36]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'PostalCode'}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [37]:
print("Dataframe shape: ", df.shape)

Dataframe shape:  (103, 5)


___
<div id="#explore">
    <h2><b>Explore the Data</b></h2>
</div>

___

**Getting Torontos lattitude and longitude using geopy**

In [38]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
print('The geograpical coordinate of Toronto are {}, {}.'.format(
    location.latitude, location.longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Creating a map of Toronto using folium**

In [39]:
map_toronto = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=10)
map_toronto

**Superimposing the neighborhoods on Toronto's map**

Markers are added to the map for each of the neighborhoods and each marker is labeled in the format: "PostalCode (Neighborhood), Borough"

In [41]:
for _, row in df.iterrows():
    label = '{} ({}), {}'.format(
        row.PostalCode, row.Neighborhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row.Latitude, row.Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

Limit data set to boroughs with term 'Toronto'

For this lab, we will just pick neighborhoods in boroughs that contain the word "Toronto" in them.

In [42]:
# Create dataframe with boroughs containing the term 'Toronto'
toronto_data = df[df.Borough.str.contains('Toronto')]
toronto_data.reset_index(inplace=True, drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [43]:
toronto_data.shape

(39, 5)

So, there are 39 neighborhoods with boroughs containing the term 'Toronto'

Let us visualize them on Toronto's map.

In [44]:
# Create a map instance
map_toronto = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=11)

# Add markers for all 38 neighborhoods
for _, row in toronto_data.iterrows():
    label = '{} ({}), {}'.format(
        row.PostalCode, row.Neighborhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row.Latitude, row.Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto